In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
import cupy as cp
import numpy as np
import logging

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense,Concatenate ,AveragePooling1D, Conv1D, MaxPooling1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation,Dropout


In [ ]:
EPOCHS = 20
plain_bits = 64
key_bits = 80
word_size = 4

NUM_SAMPLES = 10_000_000
NUM_VAL_SAMPLES = 1_000_000
BATCHSIZE = 5000
ABORT_TRAINING_BELOW_ACC = 0.50

## PRESENT algo CUPY

In [ ]:
import cupy as cp


def WORD_SIZE():
    return 64

Sbox = cp.array([0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd,
                 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2], dtype=cp.uint8)
PBox = cp.array([0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51,
                4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55,
                8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
                12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63], dtype=cp.uint8)


def SB(arr):
    num_words = arr.shape[1] // 4
    S = arr.copy()
    for i in range(num_words):
        to_sub = cp.zeros(arr.shape[0], dtype=cp.uint8)
        for j in range(4):
            pos = 4 * i + j
            to_sub += 2 ** (3 - j) * arr[:, pos]
        Sbox_sub = Sbox[to_sub]
        # unpackbits chỉ nhận mảng 1 chiều, kết quả 8 bit cho từng phần tử
        bits = cp.unpackbits(Sbox_sub)  # shape: (N*8,)
        bits = bits.reshape(-1, 8)[:, -4:]  # lấy đúng 4 bit cuối cho mỗi phần tử
        S[:, 4*i:4*(i+1)] = bits
    return S

def P(arr):
    arr = arr.copy()
    arr[:, PBox] = arr[:, cp.arange(64)]
    return arr

def expand_key(k, t):
    ks = [0 for _ in range(t)]
    key = k.copy()
    for r in range(t):
        ks[r] = key[:, :64].copy()
        key = cp.roll(key, 19, axis=1)
        key[:, :4] = SB(key[:, :4])
        # XOR round counter
        round_bin = cp.unpackbits(cp.array([r+1], dtype=cp.uint8))
        # Điều chỉnh shape để xor vào key slice [-23:-15] với round_bin[-8:]
        key[:, -23:-15] ^= round_bin[-8:]
    return ks

def encrypt(p, k, r):
    ks = expand_key(k, r)
    c = p.copy()
    for i in range(r-1):
        c ^= ks[i]
        c = SB(c)
        c = P(c)
    return c ^ ks[-1]

def convert_to_binary(arr):
    # arr: (num_blocks, num_words), mỗi word_size=64
    X = cp.zeros((len(arr) * WORD_SIZE(), len(arr[0])), dtype=cp.uint8)
    for i in range(len(arr) * WORD_SIZE()):
        index = i // WORD_SIZE()
        offset = WORD_SIZE() - (i % WORD_SIZE()) - 1
        X[i] = (arr[index] >> offset) & 1
    X = X.transpose()
    return X

def convert_from_binary(arr, _dtype=cp.uint64):
    num_words = arr.shape[1] // WORD_SIZE()
    X = cp.zeros((len(arr), num_words), dtype=_dtype)
    for i in range(num_words):
        for j in range(WORD_SIZE()):
            pos = WORD_SIZE() * i + j
            X[:, i] += (1 << (WORD_SIZE() - 1 - j)) * arr[:, pos].astype(cp.uint64)
    return X

def check_testvector_gpu():
    p = cp.zeros((1, 64), dtype=cp.uint8)
    k = cp.zeros((1, 80), dtype=cp.uint8)
    C = convert_from_binary(encrypt(p, k, 32))
    Chex = hex(int(C[0][0]))
    expected = '0x5579c1387b228445'
    print("Computed:", Chex)
    print("Expected:", expected)
    assert Chex == expected

# ĐỂ TEST:
check_testvector_gpu()

# Mã hóa số lượng lớn, ví dụ:
def benchmark_gpu(batch_size=100000):
    # Sinh batch plaintext/ciphertext ngẫu nhiên hoặc zero
    p = cp.zeros((batch_size, 64), dtype=cp.uint8)
    k = cp.zeros((batch_size, 80), dtype=cp.uint8)
    cp.cuda.Stream.null.synchronize()
    import time
    st = time.time()
    C = encrypt(p, k, 32)
    cp.cuda.Stream.null.synchronize()
    et = time.time()
    print(f'{batch_size} blocks in {et-st:.4f}s, tốc độ = {batch_size/(et-st):.2f} blocks/sec.')

# benchmark_gpu(10000)

Computed: 0x5579c1387b228445
Expected: 0x5579c1387b228445


## Neural distinguisher

In [ ]:
# From an integer, returns a 1 X num_bits numpy uint8 array
def integer_to_binary_array(int_val, num_bits):
    return cp.array([int(i) for i in bin(int_val)[2:].zfill(num_bits)], dtype = cp.uint8).reshape(1, num_bits)


### Choose model by running model cell, included make_train_data

#### Make LBEC

In [ ]:
# # Make LBEC
# def make_model(input_size, bits_per_token=8, vocab_size=256, embed_dim=64, lstm_units=128, dense_units=[128, 64], l2_reg=0.01, dropout_rate=0.2):
#     from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, BatchNormalization, Dropout
#     from tensorflow.keras.models import Model
#     seq_len = input_size // bits_per_token
#     if input_size % bits_per_token != 0:
#         raise ValueError(f"input_size ({input_size}) must be divisible by bits_per_token ({bits_per_token})")
#     input_layer = Input(shape=(24,), dtype='int32')
#     x = Embedding(input_dim=vocab_size, output_dim=embed_dim)(input_layer)
#     x = LSTM(lstm_units, return_sequences=True, recurrent_dropout=0.2, kernel_regularizer=regularizers.l2(l2_reg))(x)
#     lstm_out, state_h, state_c = LSTM(lstm_units, return_state=True, recurrent_dropout=0.2, kernel_regularizer=regularizers.l2(l2_reg))(x)
#     context_vector = Concatenate()([state_h, state_c])
#     x = BatchNormalization()(context_vector)
#     x = Dense(dense_units[0], activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
#     x = BatchNormalization()(x)
#     x = Dropout(dropout_rate)(x)
#     x = Dense(dense_units[1], activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
#     x = BatchNormalization()(x)
#     x = Dropout(dropout_rate)(x)
#     output = Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_reg))(x)
#     model = Model(inputs=input_layer, outputs=output)
#     return model
# make_lbecnet = make_model

# # def make_train_data_sequence(encryption_function, plain_bits, key_bits, n, nr,
# #                                     delta_state=0, bits_per_token=8, seq_len=None, batch_size=10**6):
# #     """
# #     Sinh dữ liệu cho sequence-based neural distinguisher đúng theo Algorithm 1 (Sequence Present).
# #     Output: X (sequence of tokens), Y (label: 0=random diff, 1=real diff)
# #     """
# #     import cupy as cp

# #     X_batches, Y_batches = [], []
# #     num_batches = (n + batch_size - 1) // batch_size

# #     for i in range(num_batches):
# #         curr_n = min(batch_size, n - i * batch_size)

# #         # Step 1–4: random key, plaintext P1, encrypt P1 → C1
# #         keys = (cp.frombuffer(cp.random.bytes(curr_n * key_bits), dtype=cp.uint8) & 1).reshape(curr_n, key_bits)
# #         P1 = (cp.frombuffer(cp.random.bytes(curr_n * plain_bits), dtype=cp.uint8) & 1).reshape(curr_n, plain_bits)
# #         C1 = encryption_function(P1, keys, nr)

# #         # Step 6–9: Random 0/1 choice → tạo P2 và Label tương ứng
# #         Y = cp.random.randint(0, 2, size=(curr_n,), dtype=cp.uint8)
# #         P2 = cp.empty_like(P1)

# #         # Nếu Y == 1: dùng P2 = P1 ⊕ ΔP
# #         real_indices = cp.where(Y == 1)[0]
# #         if isinstance(delta_state, int):
# #             delta = cp.zeros_like(P1)
# #             delta += delta_state
# #         else:
# #             delta = cp.array(delta_state, dtype=cp.uint8)
# #         P2[real_indices] = P1[real_indices] ^ delta[real_indices]

# #         # Nếu Y == 0: P2 là ngẫu nhiên
# #         rand_indices = cp.where(Y == 0)[0]
# #         P2[rand_indices] = (cp.frombuffer(cp.random.bytes(len(rand_indices) * plain_bits), dtype=cp.uint8) & 1).reshape(len(rand_indices), plain_bits)

# #         # Encrypt P2 → C2
# #         C2 = encryption_function(P2, keys, nr)

# #         # Step 15: C1 ⊕ C2
# #         CXOR = C1 ^ C2

# #         # Step 16: concatenate C1 || C2 || C1⊕C2 → sequence input
# #         C = cp.concatenate([C1, C2, CXOR], axis=1)  # shape = (curr_n, total_bits)
# #         total_bits = C.shape[1]
# #         bits_per_token =8
# #         # Step: reshape thành tokens (mỗi token là 1 byte = 8 bits)
# #         if bits_per_token == 0 or total_bits % bits_per_token != 0:
# #             raise ValueError("bits_per_token must divide total_bits")

# #         max_tokens = total_bits // bits_per_token
# #         final_seq_len = max_tokens if seq_len is None else seq_len
# #         if final_seq_len > max_tokens:
# #             raise ValueError("seq_len exceeds maximum")

# #         C = C[:, :final_seq_len * bits_per_token].reshape(curr_n, final_seq_len, bits_per_token).astype(cp.uint8)

# #         # Convert mỗi token (8 bits) → số nguyên 0–255
# #         powers = cp.array(2 ** cp.arange(bits_per_token - 1, -1, -1), dtype=cp.uint8)
# #         X_seq = (C * powers[None, None, :]).sum(axis=2).astype(cp.int32)

# #         # Append batch
# #         X_batches.append(X_seq)
# #         Y_batches.append(Y)

# #     X_out = cp.concatenate(X_batches, axis=0)
# #     Y_out = cp.concatenate(Y_batches, axis=0)
# #     return X_out, Y_out

# def make_train_data_sequence(encryption_function, plain_bits, key_bits, n, nr,
#                                        delta_state=0, bits_per_token=8, seq_len=None, batch_size=10**6):
#     """
#     Sinh dữ liệu cho sequence-based neural distinguisher theo Algorithm 1 (Sequence Present),
#     nhưng sử dụng một khóa cố định (fixed-key) cho toàn bộ dataset.
#     Output: X (sequence of tokens), Y (label: 0=random diff, 1=real diff)
#     """
#     import cupy as cp

#     # === Generate fixed key cho tất cả samples
#     fixed_key = (cp.frombuffer(cp.random.bytes(key_bits), dtype=cp.uint8) & 1).reshape(1, key_bits)

#     X_batches, Y_batches = [], []
#     num_batches = (n + batch_size - 1) // batch_size

#     for i in range(num_batches):
#         curr_n = min(batch_size, n - i * batch_size)

#         # Lặp lại fixed_key cho toàn batch
#         keys = cp.repeat(fixed_key, curr_n, axis=0)

#         # Random plaintexts P1
#         P1 = (cp.frombuffer(cp.random.bytes(curr_n * plain_bits), dtype=cp.uint8) & 1).reshape(curr_n, plain_bits)
#         C1 = encryption_function(P1, keys, nr)

#         # Sinh nhãn (0=random, 1=real)
#         Y = cp.random.randint(0, 2, size=(curr_n,), dtype=cp.uint8)
#         P2 = cp.empty_like(P1)

#         # P2 = P1 ⊕ ΔP nếu label = 1
#         real_indices = cp.where(Y == 1)[0]
#         if isinstance(delta_state, int):
#             delta = cp.zeros_like(P1)
#             delta += delta_state
#         else:
#             delta = cp.array(delta_state, dtype=cp.uint8)
#         P2[real_indices] = P1[real_indices] ^ delta[real_indices]

#         # P2 random nếu label = 0
#         rand_indices = cp.where(Y == 0)[0]
#         P2[rand_indices] = (cp.frombuffer(cp.random.bytes(len(rand_indices) * plain_bits), dtype=cp.uint8) & 1).reshape(len(rand_indices), plain_bits)

#         # Encrypt P2 → C2
#         C2 = encryption_function(P2, keys, nr)

#         # Ghép C1 || C2 || C1⊕C2
#         CXOR = C1 ^ C2
#         C = cp.concatenate([C1, C2, CXOR], axis=1)
#         total_bits = C.shape[1]
#         bits_per_token =8
#         # Chuyển thành sequence of tokens
#         if bits_per_token == 0 or total_bits % bits_per_token != 0:
#             raise ValueError("bits_per_token must divide total_bits")

#         max_tokens = total_bits // bits_per_token
#         final_seq_len = max_tokens if seq_len is None else seq_len
#         if final_seq_len > max_tokens:
#             raise ValueError("seq_len exceeds maximum")

#         C = C[:, :final_seq_len * bits_per_token].reshape(curr_n, final_seq_len, bits_per_token).astype(cp.uint8)

#         # Convert bits → token (số nguyên 0–255)
#         powers = cp.array(2 ** cp.arange(bits_per_token - 1, -1, -1), dtype=cp.uint8)
#         X_seq = (C * powers[None, None, :]).sum(axis=2).astype(cp.int32)

#         X_batches.append(X_seq)
#         Y_batches.append(Y)

#     X_out = cp.concatenate(X_batches, axis=0)
#     Y_out = cp.concatenate(Y_batches, axis=0)
#     return X_out, Y_out



#### Make DbitNET

In [ ]:
# # Make DbitNET

# def make_model(input_size=64, n_filters=32, n_add_filters=16):
#     """Create a DBITNet model with GA-CAM integrated."""
#     import tensorflow as tf
#     from tensorflow.keras.models import Model
#     from tensorflow.keras.layers import (
#         Input, Conv1D, Dense, Lambda, BatchNormalization,
#         Activation, Add, GlobalAveragePooling1D, GlobalMaxPooling1D,
#         Reshape, Multiply, Concatenate, Flatten
#     )
#     from tensorflow.keras.regularizers import l2

#     def ga_cam_block(x, ratio=16):
#         channels = x.shape[-1]
#         avg_pool = GlobalAveragePooling1D()(x)
#         max_pool = GlobalMaxPooling1D()(x)

#         avg_pool = Reshape((1, channels))(avg_pool)
#         max_pool = Reshape((1, channels))(max_pool)

#         shared = tf.keras.Sequential([
#             Conv1D(filters=channels // ratio, kernel_size=1, activation='relu', use_bias=False),
#             Conv1D(filters=channels, kernel_size=1, use_bias=False)
#         ])

#         avg_out = shared(avg_pool)
#         max_out = shared(max_pool)
#         scale = Activation('sigmoid')(Add()([avg_out, max_out]))

#         return Multiply()([x, scale])

#     # determine the dilation rates from the given input size
#     def get_dilation_rates(size):
#         rates = []
#         while size >= 8:
#             rates.append(int(size / 2 - 1))
#             size = size // 2
#         return rates

#     dilation_rates = get_dilation_rates(input_size)

#     # prediction head parameters
#     d1 = 256
#     d2 = 64
#     reg_param = 1e-5

#     # input
#     inputs = Input(shape=(input_size, 1))
#     x = Lambda(lambda x: (x - 0.5) / 0.5)(inputs)  # normalize to [-1, 1]

#     for dilation_rate in dilation_rates:
#         # Wide-narrow + GA-CAM block
#         x = Conv1D(filters=n_filters, kernel_size=2, padding='valid',
#                    dilation_rate=dilation_rate, strides=1, activation='relu')(x)
#         x = BatchNormalization()(x)
#         x = ga_cam_block(x)

#         # Narrow causal conv + residual
#         x_skip = x
#         x = Conv1D(filters=n_filters, kernel_size=2, padding='causal',
#                    dilation_rate=1, activation='relu')(x)
#         x = Add()([x, x_skip])
#         x = BatchNormalization()(x)

#         n_filters += n_add_filters

#     # prediction head
#     out = Flatten()(x)
#     out = Dense(d1, kernel_regularizer=l2(reg_param))(out)
#     out = BatchNormalization()(out)
#     out = Activation('relu')(out)
#     out = Dense(d1, kernel_regularizer=l2(reg_param))(out)
#     out = BatchNormalization()(out)
#     out = Activation('relu')(out)
#     out = Dense(d2, kernel_regularizer=l2(reg_param))(out)
#     out = BatchNormalization()(out)
#     out = Activation('relu')(out)
#     out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(out)

#     model = Model(inputs, out)
#     return model


# make_dbitnet = make_model

# import cupy as cp

# def make_train_data(encryption_function, plain_bits, key_bits, n, nr, delta_state=0, delta_key=0, batch_size=10**6):
#     """Sinh dữ liệu NDC dạng [C0 || C1], nhãn Y ∈ {0,1}, chia batch để tránh OOM."""
#     X_batches, Y_batches = [], []
#     num_batches = (n + batch_size - 1) // batch_size

#     for i in range(num_batches):
#         curr_n = min(batch_size, n - i * batch_size)

#         keys0 = cp.random.randint(0, 2, (curr_n, key_bits), dtype=cp.uint8)
#         pt0 = cp.random.randint(0, 2, (curr_n, plain_bits), dtype=cp.uint8)

#         if isinstance(delta_key, int):
#             dk = cp.full((curr_n, key_bits), delta_key, dtype=cp.uint8)
#         else:
#             dk = cp.array(delta_key, dtype=cp.uint8)
#             if dk.shape[0] != curr_n:
#                 dk = cp.tile(dk, (curr_n, 1))

#         if isinstance(delta_state, int):
#             ds = cp.full((curr_n, plain_bits), delta_state, dtype=cp.uint8)
#         else:
#             ds = cp.array(delta_state, dtype=cp.uint8)
#             if ds.shape[0] != curr_n:
#                 ds = cp.tile(ds, (curr_n, 1))

#         keys1 = keys0 ^ dk
#         pt1 = pt0 ^ ds

#         C0 = encryption_function(pt0, keys0, nr)
#         C1 = encryption_function(pt1, keys1, nr)
#         C = cp.hstack([C0, C1])

#         Y = cp.random.randint(0, 2, (curr_n,), dtype=cp.uint8)
#         rand_idx = cp.where(Y == 0)[0]

#         if rand_idx.size > 0:
#             total_cipher_bits = C.shape[1]
#             rand_c = cp.random.randint(0, 2, (rand_idx.size, total_cipher_bits), dtype=cp.uint8)
#             C[rand_idx] = rand_c

#         X_batches.append(C)
#         Y_batches.append(Y)

#     X_all = cp.concatenate(X_batches, axis=0)
#     Y_all = cp.concatenate(Y_batches, axis=0)
#     return X_all, Y_all


#### Make Inception

Attention Modules: ECA

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D, Conv1D, Multiply, Reshape, Activation

def ECA_Module(input_tensor, gamma=2, b=1):
    """
    Efficient Channel Attention (ECA)
    input_tensor: shape (B, L, C) or (B, C), typically output of a Conv1D
    returns: same shape, with channel-wise attention applied
    """
    channel = input_tensor.shape[-1]

    # Step 1: Global Average Pooling
    gap = GlobalAveragePooling1D()(input_tensor)  # shape: (B, C)

    # Step 2: Adaptive kernel size
    t = int(abs((tf.math.log(tf.cast(channel, tf.float32)) / tf.math.log(2.0)) / gamma + b))
    k_size = t if t % 2 == 1 else t + 1  # ensure odd kernel size

    # Step 3: Conv1D across channels (temporal domain)
    x = Reshape((channel, 1))(gap)  # shape: (B, C, 1)
    x = Conv1D(1, kernel_size=k_size, padding='same', use_bias=False)(x)  # shape: (B, C, 1)
    x = Activation('sigmoid')(x)

    # Step 4: Reweight input_tensor
    x = Reshape((1, channel))(x)  # shape: (B, 1, C)
    output = Multiply()([input_tensor, x])  # broadcasting

    return output


In [ ]:
# Inception
# #make residual tower of convolutional blocks

def make_model_inception_present80(pairs = 2,word_size = 16):
    """
    Inception-based Neural Distinguisher cho PRESENT-80
    Cố định các tham số đặc trưng của PRESENT: word_size = 16, pairs = 2
    Input shape: (192,) tương ứng với 2 cặp × 6 thành phần × 16 bit
    """
    import tensorflow as tf
    from tensorflow.keras.layers import (Input, Reshape, Permute, Conv1D,
                                        Concatenate, BatchNormalization, Activation,
                                        Add, Flatten, Dropout, Dense)
    from tensorflow.keras.models import Model
    from tensorflow.keras.regularizers import l2

    # Cấu hình cố định

    input_dim = pairs * 6 * word_size  # = 192
    num_filters = 32
    d1, d2 = 64, 64
    depth = 5
    ks = 3
    reg_param = 1e-4

    inp = Input(shape=(input_dim,))
    x = Reshape((pairs, 6, word_size))(inp)     # (None, 2, 6, 16)
    x = Permute((1, 3, 2))(x)                   # (None, 2, 16, 6)
    x = Reshape((pairs * word_size, 6))(x)      # (None, 32, 6)

    # Inception block
    conv01 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(x)
    conv02 = Conv1D(num_filters, kernel_size=3, padding='same', kernel_regularizer=l2(reg_param))(x)
    conv03 = Conv1D(num_filters, kernel_size=5, padding='same', kernel_regularizer=l2(reg_param))(x)
    conv04 = Conv1D(num_filters, kernel_size=7, padding='same', kernel_regularizer=l2(reg_param))(x)
    x = Concatenate(axis=-1)([conv01, conv02, conv03, conv04])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # ✅ ECA Attention sau Inception block
    x = ECA_Module(x)

    shortcut = x

    # Residual tower
    for _ in range(depth):
        conv1 = Conv1D(num_filters * 4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(num_filters * 4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])
        ks += 2

    # Prediction head
    x = Flatten()(shortcut)
    x = Dropout(0.3)(x)
    x = Dense(512, kernel_regularizer=l2(reg_param))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(d1, kernel_regularizer=l2(reg_param))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(d2, kernel_regularizer=l2(reg_param))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(x)

    return Model(inputs=inp, outputs=out)



In [ ]:
import cupy as cp

import numpy as np

def make_train_data_multi_pair(encryption_function, plain_bits, key_bits, n, nr,
                               delta_state=0, delta_key=0, pairs=2, batch_size=10**6):
    """
    Sinh dữ liệu huấn luyện cho mô hình Neural Distinguisher sử dụng nhiều cặp ciphertext.
    Output: X shape (n, pairs * 6 * word_size), Y shape (n,)
    """

    X_batches, Y_batches = [], []
    word_size = plain_bits // 2
    num_blocks = 3  # C0, C1, invPerm(C0), invPerm(C1), diff, invPermDiff
    input_dim = pairs * num_blocks * word_size

    num_batches = (n + batch_size - 1) // batch_size

    for _ in range(num_batches):
        curr_n = min(batch_size, n - len(Y_batches) * batch_size)
        Y = cp.random.randint(0, 2, (curr_n,), dtype=cp.uint8)
        Y_tile = cp.tile(Y, pairs)

        # Khởi tạo plaintext & key
        pt0 = cp.random.randint(0, 2, (curr_n * pairs, plain_bits), dtype=cp.uint8)
        key0 = cp.random.randint(0, 2, (curr_n * pairs, key_bits), dtype=cp.uint8)

        pt1 = pt0 ^ delta_state
        key1 = key0 ^ delta_key

        C0 = encryption_function(pt0, key0, nr)
        C1 = encryption_function(pt1, key1, nr)

        # Gán nhãn random: thay thế C0, C1 bằng ngẫu nhiên nếu nhãn = 0
        mask = Y_tile == 0
        num_false = int(mask.sum().item())

        if num_false > 0:
            C0[mask] = cp.random.randint(0, 2, (num_false, plain_bits), dtype=cp.uint8)
            C1[mask] = cp.random.randint(0, 2, (num_false, plain_bits), dtype=cp.uint8)

        # Feature engineering: [C0 || C1 || C0^C1]
        diff = C0 ^ C1
        X = cp.concatenate([C0, C1, diff], axis=1)  # shape (n * pairs, 3 * word_size * 2)

        # reshape to (curr_n, input_dim)
        X = X.reshape(curr_n, -1)
        X_batches.append(cp.asnumpy(X))
        Y_batches.append(cp.asnumpy(Y))

    X_final = np.concatenate(X_batches, axis=0)
    Y_final = np.concatenate(Y_batches, axis=0)

    return X_final, Y_final

In [ ]:
# Data generator
import numpy as np
import cupy as cp
from tensorflow.keras.utils import Sequence

from tensorflow.keras.utils import Sequence
import numpy as np
import cupy as cp

from tensorflow.keras.utils import Sequence
import numpy as np
import cupy as cp

class NDCMultiPairGenerator(Sequence):
    def __init__(self, encryption_function, plain_bits, key_bits, nr,
                 delta_state=0, delta_key=0, n_samples=10**7, batch_size=10**5,
                 pairs=2, use_gpu=True, to_float32=True):
        """
        Generator sinh dữ liệu cho Neural Distinguisher (related-key + multi-pair)
        Format input: [ΔC || C || C*] với ΔC = C ^ C*
        """
        self.encryption_function = encryption_function
        self.plain_bits = plain_bits
        self.key_bits = key_bits
        self.nr = nr
        self.delta_state = delta_state
        self.delta_key = delta_key
        self.n = n_samples
        self.batch_size = batch_size
        self.pairs = pairs
        self.use_gpu = use_gpu
        self.to_float32 = to_float32

        self.steps = (self.n + self.batch_size - 1) // self.batch_size
        self.input_dim = self.pairs * 3 * self.plain_bits  # ΔC || C || C*

    def __len__(self):
        return self.steps

    def __getitem__(self, idx):
        curr_n = min(self.batch_size, self.n - idx * self.batch_size)
        lib = cp if self.use_gpu else np

        # Ép kiểu delta về đúng backend
        delta_key = self.delta_key
        delta_state = self.delta_state
        if self.use_gpu:
            if isinstance(delta_key, np.ndarray): delta_key = cp.asarray(delta_key)
            if isinstance(delta_state, np.ndarray): delta_state = cp.asarray(delta_state)
        else:
            if isinstance(delta_key, cp.ndarray): delta_key = cp.asnumpy(delta_key)
            if isinstance(delta_state, cp.ndarray): delta_state = cp.asnumpy(delta_state)

        # Sinh nhãn cân bằng
        Y = lib.zeros(curr_n, dtype=lib.uint8)
        Y[:curr_n // 2] = 1
        lib.random.shuffle(Y)

        # Sinh plaintext và key ngẫu nhiên
        P = lib.random.randint(0, 2, (curr_n * self.pairs, self.plain_bits), dtype=lib.uint8)
        K = lib.random.randint(0, 2, (curr_n * self.pairs, self.key_bits), dtype=lib.uint8)

        P_star = lib.empty_like(P)
        K_star = lib.empty_like(K)

        # Tạo mask theo nhãn
        mask_true = lib.repeat(Y == 1, self.pairs)
        mask_false = lib.repeat(Y == 0, self.pairs)

        # Nếu Y = 1 → đúng cặp (P*, K*)
        P_star[mask_true] = P[mask_true] ^ delta_state
        K_star[mask_true] = K[mask_true] ^ delta_key

        # Nếu Y = 0 → random noise
        P_star[mask_false] = lib.random.randint(0, 2, (int(mask_false.sum()), self.plain_bits), dtype=lib.uint8)
        K_star[mask_false] = lib.random.randint(0, 2, (int(mask_false.sum()), self.key_bits), dtype=lib.uint8)

        # Mã hóa
        C = self.encryption_function(P, K, self.nr)
        C_star = self.encryption_function(P_star, K_star, self.nr)
        delta_C = C ^ C_star

        # Format [ΔC || C || C*]
        X = lib.concatenate([delta_C, C, C_star], axis=1)
        X = X.reshape(curr_n, -1)

        if self.to_float32:
            X = X.astype(lib.float32)

        if self.use_gpu:
            X = cp.asnumpy(X)
            Y = cp.asnumpy(Y)

        return X, Y


### Training phase (simplified)

In [ ]:
## Callbacks
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ModelCheckpoint, LearningRateScheduler, LambdaCallback,
    EarlyStopping, TerminateOnNaN, TensorBoard
)
import os
import datetime

# === CYCLIC LR ===
def cyclic_lr(num_epochs, high_lr, low_lr):
    return lambda i: low_lr + ((num_epochs - 1) - i % num_epochs) / (num_epochs - 1) * (high_lr - low_lr)

# === PRINT CURRENT LR ===
def print_lr(epoch, logs):
    lr = tf.keras.backend.get_value(model.optimizer.learning_rate)
    print(f"Epoch {epoch}: Learning Rate = {lr:.6f}")

lr_logger = LambdaCallback(on_epoch_end=print_lr)
lr_scheduler = LearningRateScheduler(cyclic_lr(num_epochs=10, high_lr=0.002, low_lr=0.0001))

# === MODEL CHECKPOINT ===
checkpoint_cb = ModelCheckpoint(
    filepath='best_model.h5',        # hoặc '/content/best_model.h5' nếu dùng Colab
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# === EARLY STOPPING ===
earlystop_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# === TERMINATE ON NaN ===
terminate_nan_cb = TerminateOnNaN()

# === TENSORBOARD ===
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)

# === COMBINE ALL CALLBACKS ===
callbacks = [
    checkpoint_cb,
    lr_scheduler,
    lr_logger,
    earlystop_cb,
    terminate_nan_cb,
    tensorboard_cb
]


In [ ]:
def update_checkpoint_in_callbacks(callbacks, rounds, save_dir='checkpoints'):
    """
    Cập nhật ModelCheckpoint callback trong list callbacks theo số rounds.
    """
    import os
    from tensorflow.keras.callbacks import ModelCheckpoint

    os.makedirs(save_dir, exist_ok=True)
    filename = f"{save_dir}/best_model_{rounds}rounds.h5"
    new_ckpt = ModelCheckpoint(
        filepath=filename,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )

    # Tìm và thay thế phần tử là ModelCheckpoint
    for i, cb in enumerate(callbacks):
        if isinstance(cb, ModelCheckpoint):
            callbacks[i] = new_ckpt
            break
    else:
        # Nếu chưa có, thêm vào
        callbacks.append(new_ckpt)

    return callbacks


#### Selecting delta_key

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

def select_best_delta_key(
    encryption_function, input_difference,
    plain_bits, key_bits, n_round, pairs,
    n_samples=100_000, batch_size=5000, use_gpu=True
):
    delta_plain = integer_to_binary_array(input_difference, plain_bits)
    best_score = -1.0
    best_bit = -1
    all_scores = {}

    print("🔍 Searching for best delta_key (Hamming weight = 1):")

    for bit in range(key_bits):
        delta_key = np.zeros(key_bits, dtype=cp.uint8)
        delta_key[bit] = 1

        gen = NDCMultiPairGenerator(
            encryption_function=encryption_function,
            plain_bits=plain_bits, key_bits=key_bits, nr=n_round,
            delta_state=delta_plain,
            delta_key=delta_key,
            pairs=pairs,
            n_samples=n_samples, batch_size=batch_size,
            use_gpu=use_gpu, to_float32=True
        )

        X_val, Y_val = gen[0]

        try:
            pca = PCA(n_components=2).fit_transform(X_val)
            score = silhouette_score(pca, Y_val)
        except Exception as e:
            score = -1.0  # fallback nếu lỗi
        all_scores[bit] = score

        # print(f"∆K bit {bit:02d} → silhouette score = {score:.5f}")
        if score > best_score:
            best_score = score
            best_bit = bit

    print(f"\n✅ Best delta_key bit: {best_bit} with score = {best_score:.5f}")
    return best_bit, best_score, all_scores


#### Training

In [ ]:
# === Config ===
input_difference = 0x00000080
n_round = 7
pairs = 8
plain_bits = 64
key_bits = 80
batch_size = 5000
val_batch_size = 20_000
EPOCHS = 20
callbacks = update_checkpoint_in_callbacks(callbacks, rounds=n_round)

# === ∆P and ∆K ===

#Chọn best_bit cho delta_key
best_bit, best_score, all_scores = select_best_delta_key(
    encryption_function=encrypt,
    input_difference=input_difference,
    plain_bits=plain_bits,
    key_bits=key_bits,
    n_round=n_round,
    pairs=pairs,
    use_gpu=True
)

delta_plain = integer_to_binary_array(input_difference, plain_bits)
delta_key = np.zeros(key_bits, dtype=np.uint8)
delta_key[best_bit] = 1

# === Generator ===
gen = NDCMultiPairGenerator(
    encryption_function=encrypt,
    plain_bits=plain_bits, key_bits=key_bits, nr=n_round,
    delta_state=delta_plain,
    delta_key=delta_key,
    pairs=pairs,
    n_samples=10**7, batch_size=batch_size,
    use_gpu=True
)

gen_val = NDCMultiPairGenerator(
    encryption_function=encrypt,
    plain_bits=plain_bits, key_bits=key_bits, nr=n_round,
    delta_state=delta_plain,
    delta_key=delta_key,
    pairs=pairs,
    n_samples=10**6, batch_size=val_batch_size,
    use_gpu=True
)
X_val, Y_val = gen[0]
pca = PCA(n_components=2).fit_transform(X_val)
score = silhouette_score(pca, Y_val)
print(score)

🔍 Searching for best delta_key (Hamming weight = 1):

✅ Best delta_key bit: 56 with score = 0.47582
0.4787769


In [ ]:
# Chuyển mảng nhị phân thành chuỗi hex
def binary_array_to_hex_with_0x(arr):
    # Đảm bảo độ dài chia hết cho 8
    assert len(arr) % 8 == 0, "Độ dài delta_key phải chia hết cho 8"
    hex_str = ""
    for i in range(0, len(arr), 8):
        byte = arr[i:i+8]
        value = 0
        for bit in byte:
            value = (value << 1) | bit
        hex_str += f"{value:02x}"
    return "0x" + hex_str

In [ ]:
print(binary_array_to_hex_with_0x(delta_key))

0x00000000000000800000


In [ ]:
model = make_model_inception_present80(pairs= pairs*2)
optimizer = tf.keras.optimizers.Adam(amsgrad=True)

model.compile(optimizer=optimizer, loss='mse', metrics=['acc'])
history = model.fit(gen, epochs=10,validation_data=gen_val ,batch_size=BATCHSIZE,callbacks= callbacks,verbose=True)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

model = make_model_inception_present80(pairs= pairs*2)
optimizer = tf.keras.optimizers.Adam(amsgrad=True)
model.compile(optimizer=optimizer, loss='mse', metrics=['acc'])
model.load_weights("/content/checkpoints/best_model_9rounds.h5")

history = model.fit(gen, epochs=10,validation_data=gen_val ,batch_size=BATCHSIZE,callbacks= callbacks,verbose=True)


In [ ]:
# # # === ∆P and ∆K ===
# # delta_plain = integer_to_binary_array(input_difference, plain_bits)

# delta_key = np.zeros(key_bits, dtype=np.uint8)
# delta_key[best_bit] = 1   # ✅ Related-key

# test_gen = NDCMultiPairGenerator(
#     encryption_function=encrypt,
#     plain_bits=64, key_bits=80, nr=13,
#     delta_state=delta_plain,
#     delta_key=delta_key,
#     n_samples=1_000_000,
#     batch_size=10_000,
#     pairs=7,
#     use_gpu=True
# )
# model.evaluate(test_gen)

#### Evaluate functions

In [ ]:
def evaluate_on_fresh_test_sets(model, round_number, n_repeat=5, log_path=None):
    """
    Đánh giá mô hình trên n_repeat tập test ngẫu nhiên, mỗi tập có 1 triệu mẫu.

    Parameters:
        model: mô hình đã được huấn luyện (hoặc load lại weights).
        data_generator: hàm sinh dữ liệu (num_samples, round_number) → (X, Y).
        round_number: số round dùng để sinh dữ liệu test.
        n_repeat: số lần sinh và đánh giá (mặc định: 5).
        log_path: đường dẫn để ghi log (nếu có).

    Returns:
        avg_acc, std_acc: trung bình và độ lệch chuẩn accuracy trên các tập test.
    """
    import numpy as np

    print(f"\n📊 Evaluating model on {n_repeat} fresh test sets for round {round_number}...")


    test_accs = []
    for i in range(n_repeat):
        test_gen = NDCMultiPairGenerator(
          encryption_function=encrypt,
          plain_bits=64, key_bits=80, nr=round_number,
          delta_state=integer_to_binary_array(input_difference, plain_bits),
          delta_key=delta_key,
          n_samples=1000000,
          batch_size=10000,
          pairs=pairs,
          use_gpu=True
        )
        _, acc = model.evaluate(test_gen, verbose=0)
        print(f"  Test {i+1}: accuracy = {acc:.4f}")
        test_accs.append(acc)

    avg_acc = np.mean(test_accs)
    std_acc = np.std(test_accs)
    print(f"✅ Avg test accuracy: {avg_acc:.4f} ± {std_acc:.4f}")

    # Ghi log nếu cần
    if log_path:
        with open(log_path, 'w') as f:
            for i, acc in enumerate(test_accs):
                f.write(f'Test {i+1}: {acc:.6f}\n')
            f.write(f'\nAverage: {avg_acc:.6f}, Std: {std_acc:.6f}\n')

    return avg_acc, std_acc


In [ ]:
import numpy as np
from scipy.stats import norm
from tqdm import tqdm

def evaluate_with_statistics(model, round_number, n_repeat=20, log_path=None):
    """
    Đánh giá mô hình nhiều lần và tính thống kê để kiểm tra xem kết quả có vượt ngẫu nhiên không.

    Parameters:
        model: Mô hình đã huấn luyện.
        round_number: Số round PRESENT dùng để sinh dữ liệu test.
        n_repeat: Số lần sinh và đánh giá (khuyến nghị: 20–30).
        log_path: Nếu có, lưu log kết quả vào file.

    Returns:
        dict: Chứa avg_acc, std_acc, z_score, p_value.
    """
    print(f"\n📊 Evaluating model on {n_repeat} fresh test sets for round {round_number}...")

    test_accs = []
    for i in tqdm(range(n_repeat)):
        test_gen = NDCMultiPairGenerator(
            encryption_function=encryption_function,
            plain_bits=64, key_bits=80, nr=round_number,
            delta_state=integer_to_binary_array(input_difference, 64),
            delta_key=delta_key,
            n_samples=1000000,
            batch_size=10000,
            pairs=pairs,
            use_gpu=True
        )
        _, acc = model.evaluate(test_gen, verbose=0)
        test_accs.append(acc)

    # Tính thống kê
    accs = np.array(test_accs)
    avg_acc = np.mean(accs)
    std_acc = np.std(accs)

    # Z-score: độ lệch trung bình so với 50%
    mean_random = 0.5
    std_random = 0.0005  # Với mỗi test set 1 triệu mẫu
    std_mean = std_random / np.sqrt(n_repeat)
    z_score = (avg_acc - mean_random) / std_mean
    p_value = 1 - norm.cdf(z_score)

    print(f"\n✅ Average Accuracy: {avg_acc:.5f} ± {std_acc:.5f}")
    print(f"📐 Z-score = {z_score:.2f},  P-value = {p_value:.4e}")
    if p_value < 0.01:
        print("✨ Statistically significant improvement over random.")
    else:
        print("⚠️  Accuracy may still be due to random guessing.")

    # Ghi log nếu cần
    if log_path:
        with open(log_path, 'w') as f:
            for i, acc in enumerate(accs):
                f.write(f"Test {i+1}: {acc:.6f}\n")
            f.write(f"\nAverage: {avg_acc:.6f}, Std: {std_acc:.6f}\n")
            f.write(f"Z-score: {z_score:.2f}, P-value: {p_value:.4e}\n")

    return {
        'avg_acc': avg_acc,
        'std_acc': std_acc,
        'z_score': z_score,
        'p_value': p_value
    }


### Load and eval models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import load_model

n_round = 7
pairs= 8
model = make_model_inception_present80(pairs= pairs*2)
model.compile(optimizer=optimizer, loss='mse', metrics=['acc'])

# #Chọn best_bit cho delta_key
# best_bit, best_score, all_scores = select_best_delta_key(
#     encryption_function=encrypt,
#     input_difference=input_difference,
#     plain_bits=64,
#     key_bits=80,
#     n_round=8,
#     pairs=8,
#     use_gpu=True
# )
delta_plain = integer_to_binary_array(input_difference, plain_bits)
delta_key = np.zeros(key_bits, dtype=np.uint8)
delta_key[56] = 1


model.load_weights("/content/checkpoints/best_model_7rounds.h5")

evaluate_on_fresh_test_sets(model,round_number= n_round, log_path='')


In [ ]:
evaluate_with_statistics(model, round_number=n_round, n_repeat=20, log_path=None)


In [ ]:
evaluate_with_statistics(model, round_number=12, n_repeat=20, log_path=None)
